#### Dermatology Model (WIP)

Descrição: 

In [1]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


In [2]:
# --- 1. Carregar e Inspecionar o Dataset ---
print("--- 1. Carregando e Inspecionando o Dataset Dermatology ---")
car_evaluation = fetch_ucirepo(id=33)
X_original = car_evaluation.data.features
y_original = car_evaluation.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")
print("-" * 50)

--- 1. Carregando e Inspecionando o Dataset Dermatology ---
Dimensão inicial de X: (366, 34)
Colunas de X e seus tipos de dados:
erythema                                         int64
scaling                                          int64
definite-borders                                 int64
itching                                          int64
koebner phenomenon                               int64
polygonal papules                                int64
follicular papules                               int64
oral-mucosal involvement                         int64
knee elbow involvement                           int64
scalp involvement                                int64
family history                                   int64
melanin incontinence                             int64
eosinophils in the infiltrate                    int64
pnl infiltrate                                   int64
fibrosis of the papillary dermis                 int64
exocytosis                                    

In [ ]:
from module import termometro
print("--- 3. Binarização dos Atributos para o WiSARD ---")
X_encoded_parts = []
n_bits = 8 

for col in X_original.columns:
    X_encoded_parts.append(termometro.codificador_termometro(X_original[col].astype(float), n_bits))
    #print(f"Coluna '{col}' binarizada com {n_bits} bits (Termômetro).")

X_final_binary = np.hstack(X_encoded_parts)
X_encoded_uint8 = X_final_binary.astype(np.uint8)
print(X_encoded_uint8)

--- 3. Binarização dos Atributos para o WiSARD ---
[[1 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 0]]


In [9]:
# --- 3. Divisão dos Dados em Treino e Teste ---
print("--- 3. Dividindo os Dados em Treino e Teste ---")

y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")
print("-" * 50)

--- 3. Dividindo os Dados em Treino e Teste ---
Tamanho do conjunto de treino (X_train): 109 amostras, 272 bits
Tamanho do conjunto de teste (X_test): 257 amostras, 272 bits
Tipo dos elementos de X_train: uint8
--------------------------------------------------


In [11]:
# --- 4. Configuração e Treinamento do Modelo WiSARD ---
print("--- 4. Configurando e Treinando o Modelo WiSARD ---")

# addressSize (tuple_size) para o WiSARD.
# O número de bits de entrada de X_encoded_dense para Car Evaluation é 21.
# Um addressSize de 21 ainda é muito grande.
# Escolha um addressSize menor, como 2, 3 ou 4.
# Por exemplo, se X_encoded_dense tem 21 bits, e addressSize=3, teremos 21/3 = 7 RAMs por classe.
addressSize = 3 # Ajuste este valor. 2 ou 3 são bons pontos de partida.

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
print("Treinando o WiSARD...")
wsd.train(X_train, y_train)
print("Treinamento concluído.")
print("-" * 50)


# --- 5. Classificação e Avaliação do Modelo WiSARD ---
print("--- 5. Classificação e Avaliação do Modelo WiSARD ---")

print("Classificando os dados de teste...")
predictions_str = wsd.classify(X_test)
print("Classificação concluída.")

# Convertendo as previsões de string de volta para um array NumPy para fácil comparação
predictions_np = np.array(predictions_str)
y_test_np = np.array(y_test) # Converte o y_test também para numpy array para comparação elemento a elemento

print("Relatório de Classificação:")
print(classification_report(
    y_test_np, predictions_np,
    labels=np.unique(y_test)
))

--- 4. Configurando e Treinando o Modelo WiSARD ---
WiSARD inicializado com addressSize=3
Treinando o WiSARD...
Treinamento concluído.
--------------------------------------------------
--- 5. Classificação e Avaliação do Modelo WiSARD ---
Classificando os dados de teste...
Classificação concluída.
Relatório de Classificação:
training 109 of 109
classi              precision    recall  f1-score   support

           1       0.96      1.00      0.98        79
           2       0.97      0.86      0.91        43
           3       1.00      1.00      1.00        51
           4       0.84      0.94      0.89        34
           5       0.97      1.00      0.99        36
           6       1.00      0.79      0.88        14

    accuracy                           0.96       257
   macro avg       0.96      0.93      0.94       257
weighted avg       0.96      0.96      0.96       257

classifying 257 of 257 257
